In [1]:
import mediapipe as mp
import cv2
import numpy as np
import uuid 
import os

In [2]:
mp_drawing = mp.solutions.drawing_utils  ##make it easier for us to render all different landmarks in our hand 
mp_hands = mp.solutions.hands

In [3]:
# Start video capture
cap = cv2.VideoCapture(0)

# Function to count fingers correctly
def count_fingers(hand_landmarks, handedness):
    fingers = []

    # Thumb Detection: Adjust condition based on left or right hand
    thumb_tip = hand_landmarks.landmark[4]
    thumb_base = hand_landmarks.landmark[2]

    if handedness == "Right":  # Right hand
        if thumb_tip.x < thumb_base.x:  # Thumb open
            fingers.append(1)
        else:
            fingers.append(0)
    else:  # Left hand
        if thumb_tip.x > thumb_base.x:  # Thumb open
            fingers.append(1)
        else:
            fingers.append(0)

    # Other 4 fingers (Compare Tip vs. PIP joint)
    finger_tips = [8, 12, 16, 20]  # Finger tip indices
    finger_pips = [6, 10, 14, 18]  # Lower joint (PIP) indices

    for tip, pip in zip(finger_tips, finger_pips):
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[pip].y:  
            fingers.append(1)  # Finger is open
        else:
            fingers.append(0)  # Finger is closed

    return sum(fingers)  # Total number of fingers raised

# Use Mediapipe Hands Model
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=2) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR to RGB and flip horizontally
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1)
        image.flags.writeable = False

        # Process hand tracking
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        total_fingers = 0  # Reset count

        # Detect hands and count fingers
        if results.multi_hand_landmarks and results.multi_handedness:
            for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Get hand type (Left/Right)
                handedness = results.multi_handedness[idx].classification[0].label

                # Count fingers for this hand
                total_fingers += count_fingers(hand_landmarks, handedness)

        # Display the detected number
        cv2.putText(image, f"Number: {total_fingers}", (50, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

        # Show the result
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()